</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [4]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [5]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [6]:
d_f = pd.read_csv('./data/RW_FB_data_20210608.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/coverage_data_itu.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/Rwanda_Ericsson_07_06_2021.csv',low_memory=False,encoding='utf-8')
d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/rwanda_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [7]:
d_g.head(10)

,sn,source_sch,school_nam,school_cat,school_set,district,sector,cell,village,latitude,...,internet_a,num_studen,num_teache,computers,uuid,connectivi,giga_schoo,2G,3G,4G
0,1,430710.0,GS MURAMA,PUBLIC,12 YBE,RUTSIRO,MASASA,GABIRO,GABIRO,-1.973667,...,USB_MODEM,21,1,323.0,9701a58f-8724-4e63-b665-a6bc51eab7f9,3G,G_RWA_00001,3.0,3.0,3.0
1,2,521312.0,GS MUHORORO,PUBLIC,12 YBE,KARONGI,MURAMBI,MUHORORO,BWAKIRA,-2.159389,...,USB_MODEM,28,1,260.0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,3G,G_RWA_00002,3.0,NaN,3.0
2,3,120515.0,GS MWENDO,PUBLIC,12 YBE,RUHANGO,MWENDO,GAFUNZO,NYAMIGINA,-2.166806,...,USB_MODEM,43,1,217.0,ce94c79c-882e-4f64-a1d8-fcb73add4f84,3G,G_RWA_00003,3.0,3.0,3.0
3,4,120122.0,GS KIMISANGE,PUBLIC,12 YBE,KICUKIRO,KIGARAMA,BWERANKORI,NYENYERI,-1.989889,...,USB_MODEM,47,3,251.0,a80c737c-322a-4874-ab0b-ca4eb68d1507,3G,G_RWA_00004,3.0,3.0,3.0
4,5,331220.0,GS MWENDO,PUBLIC,12 YBE,GICUMBI,MUKO,MWENDO,KAGOGO,-1.730639,...,USB_MODEM,62,3,272.0,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,3G,G_RWA_00005,3.0,NaN,3.0
5,6,411305.0,GS NYAGISOZI,PUBLIC,12 YBE,NYANZA,CYABAKAMYI,KARAMA,NYABINOMBE,-2.295028,...,USB_MODEM,64,4,218.0,9fe38ae7-7d51-4646-841b-1b6daa31c987,3G,G_RWA_00006,3.0,NaN,3.0
6,7,430323.0,GS RWAMIKO,PUBLIC,12 YBE,RUTSIRO,MANIHIRA,HANIRO,KAZIRAMIHUNDE,-1.939389,...,USB_MODEM,70,2,257.0,f66b942f-034d-4061-a4f0-cb2c6771bfc2,3G,G_RWA_00007,3.0,1.0,NaN
7,8,510402.0,GS KIGARAMA,PUBLIC,12 YBE,KIREHE,KIGARAMA,KIGARAMA,SAMUKO,-2.333917,...,USB_MODEM,74,2,422.0,1daced85-eab1-416a-8270-db0ec267c179,3G,G_RWA_00008,3.0,3.0,3.0
8,9,510619.0,GS RUGANGO,PUBLIC,12 YBE,KIREHE,MUSAZA,KABUGA,RUGANGO,-2.371833,...,USB_MODEM,80,2,252.0,8b5c3930-c3ea-44d8-a510-086f8c6889aa,3G,G_RWA_00009,3.0,NaN,NaN
9,10,521042.0,GS NYABISINDU,PUBLIC,12 YBE,GATSIBO,KIRAMURUZI,NYABISINDU,AKABUGA,-1.830611,...,USB_MODEM,84,3,234.0,39bc3129-4120-4568-96d5-5220696a123b,3G,G_RWA_00010,3.0,3.0,3.0


In [8]:
# some colunm names standardization

In [9]:
d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [10]:
# cross-profiles the sources with master data for completeness and correctness

In [11]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
# for GSMA
cross_profile_gsma_master = datacompy.Compare(
master,
gsma,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='GSMA'
)
print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
print(cross_profile_gsma_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0    Master       20  4093
1  Facebook       26  4195

Column Summary
--------------

Number of columns in common: 12
Number of columns in Master but not in Facebook: 8
Number of columns in Facebook but not in Master: 14

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,091
Number of rows in Master but not in Facebook: 2
Number of rows in Facebook but not in Master: 104

Number of rows with some compared columns unequal: 8
Number of rows with all compared columns equal: 4,083

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 11
Total number of values which compare unequal: 8

Columns with Unequal Values or Types
------------------------------------

        Column Master dtype Face

In [12]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [13]:
d_e['coverage_availability'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage_availability'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage_availability'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage_availability'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_g['coverage_availability'] = np.where((d_g['2G'].isnull()) & (d_g['3G'].isnull())& (d_g['4G'].isnull()), 'NO', 'YES')

In [14]:
d_g['coverage_availability'].value_counts()

YES    4181
NO       16
Name: coverage_availability, dtype: int64

In [15]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [16]:
d_g['2G_coverage'] = np.where((d_g['2G'].isnull()), 'NO', 'YES') 
d_g['3G_coverage'] = np.where((d_g['3G'].isnull()), 'NO', 'YES') 
d_g['4G_coverage'] = np.where((d_g['4G'].isnull()), 'NO', 'YES') 

In [17]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [18]:
d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [19]:
d_e['2G_coverage'] = np.where((d_e['2G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['3G_coverage'] = np.where((d_e['3G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['4G_coverage'] = np.where((d_e['4G_coverage_quality'] != 'None'), 'YES', 'NO')
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [20]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [21]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [22]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)
d_g['coverage_type'] = d_g.apply((lambda x: coverage_type(x)),axis=1)

In [23]:
# isolating the needed colunms from the rest

In [24]:
features = ['giga_school_id','coverage_availability','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [25]:
d_fc = d_f[features]
d_ec = d_e[features]
d_it = d_i[features]
d_gs = d_g[features]

In [26]:
# more sanity checks for nan on giga_school_id

In [27]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_gs = d_gs[d_gs['giga_school_id'].notna()]

In [28]:
# visually comparing the sources for matches and mismatches across the columns

In [29]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_gs.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'GSMA','Ericsson', 'ITU'])
df_all.head()

Facebook                                        \
               coverage_availability coverage_type 2G_coverage 3G_coverage   
giga_school_id                                                               
G_RWA_00112                      YES            4G         YES         YES   
G_RWA_00116                      YES            4G         YES         YES   
G_RWA_00258                      YES            4G         YES         YES   
G_RWA_00551                      YES            4G         YES         YES   
G_RWA_00901                      YES            4G         YES         YES   

                                            GSMA                            \
               4G_coverage coverage_availability coverage_type 2G_coverage   
giga_school_id                                                               
G_RWA_00112            YES                   YES            4G         YES   
G_RWA_00116            YES                   YES            4G         YES   
G_RWA_00258            YES                   YES            4G         YES   
G_RWA_00551            YES                   YES            4G         YES   
G_RWA_00901            YES                   YES            4G         YES   

                                                    Ericsson                \
               3G_coverage 4G_coverage coverage_availability coverage_type   
giga_school_id                                                               
G_RWA_00112            YES         YES                   YES            4G   
G_RWA_00116            YES         YES                   YES            4G   
G_RWA_00258            YES         YES                   YES            4G   
G_RWA_00551            YES         YES                   YES            4G   
G_RWA_00901            YES         YES                   YES            4G   

                                                                     ITU  \
               2G_coverage 3G_coverage 4G_coverage coverage_availability   
giga_school_id                                                             
G_RWA_00112            YES         YES         YES                   YES   
G_RWA_00116            YES         YES         YES                   YES   
G_RWA_00258            YES         YES         YES                   YES   
G_RWA_00551            YES         YES         YES                   YES   
G_RWA_00901            YES         YES         YES                   YES   

                                                                  
               coverage_type 2G_coverage 3G_coverage 4G_coverage  
giga_school_id                                                    
G_RWA_00112               4G         YES         YES         YES  
G_RWA_00116               4G         YES          NO         YES  
G_RWA_00258               4G         YES          NO         YES  
G_RWA_00551               4G         YES         YES         YES  
G_RWA_00901               4G         YES          NO         YES

In [30]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage_availability                    coverage_type       \
                            Facebook GSMA Ericsson  ITU      Facebook GSMA   
giga_school_id                                                               
G_RWA_00112                      YES  YES      YES  YES            4G   4G   
G_RWA_00116                      YES  YES      YES  YES            4G   4G   
G_RWA_00258                      YES  YES      YES  YES            4G   4G   
G_RWA_00551                      YES  YES      YES  YES            4G   4G   
G_RWA_00901                      YES  YES      YES  YES            4G   4G   

                            2G_coverage                    3G_coverage       \
               Ericsson ITU    Facebook GSMA Ericsson  ITU    Facebook GSMA   
giga_school_id                                                                
G_RWA_00112          4G  4G         YES  YES      YES  YES         YES  YES   
G_RWA_00116          4G  4G         YES  YES      YES  YES         YES  YES   
G_RWA_00258          4G  4G         YES  YES      YES  YES         YES  YES   
G_RWA_00551          4G  4G         YES  YES      YES  YES         YES  YES   
G_RWA_00901          4G  4G         YES  YES      YES  YES         YES  YES   

                             4G_coverage                     
               Ericsson  ITU    Facebook GSMA Ericsson  ITU  
giga_school_id                                               
G_RWA_00112         YES  YES         YES  YES      YES  YES  
G_RWA_00116         YES   NO         YES  YES      YES  YES  
G_RWA_00258         YES   NO         YES  YES      YES  YES  
G_RWA_00551         YES  YES         YES  YES      YES  YES  
G_RWA_00901         YES   NO         YES  YES      YES  YES

In [31]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [32]:
# coverage comparison report

In [33]:
# create a separate dfs
fbc = d_fc.copy()
ecc = d_ec.copy()
ituc = d_it.copy()
gsm = d_gs.copy()
# Facebook&Ericsson
cross_profile_fb_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='Ericsson'
)
# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)
#Ericsson&ITU
cross_profile_Ericsson_itu = datacompy.Compare(
ecc,
ituc,
join_columns='giga_school_id',  
df1_name='Ericsson', 
df2_name='ITU'
)
#Ericsson&ITU
cross_profile_facebook_gsma = datacompy.Compare(
fbc,
gsm,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)

print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_gsma.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Facebook        6  4195
1  Ericsson        6  4197

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in Ericsson: 0
Number of columns in Ericsson but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,195
Number of rows in Facebook but not in Ericsson: 0
Number of rows in Ericsson but not in Facebook: 2

Number of rows with some compared columns unequal: 2,504
Number of rows with all compared columns equal: 1,691

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 5,948

Columns with Unequal Values or Types
------------------------------------

                  Col

In [34]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [35]:
#Main
import statistics as stats
df_c = pd.concat([d_fc,d_gs,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])
coverage.loc[coverage['coverage_availability'] == 'Y', 'coverage_availability'] = 'YES'
coverage.loc[coverage['coverage_availability'] == 'N', 'coverage_availability'] = 'NO'
coverage.loc[coverage['2G_coverage'] == 'Y', '2G_coverage'] = 'YES'
coverage.loc[coverage['2G_coverage'] == 'N', '2G_coverage'] = 'NO'
coverage.loc[coverage['3G_coverage'] == 'Y', '3G_coverage'] = 'YES'
coverage.loc[coverage['3G_coverage'] == 'N', '3G_coverage'] = 'NO'
coverage.loc[coverage['4G_coverage'] == 'Y', '4G_coverage'] = 'YES'
coverage.loc[coverage['4G_coverage'] == 'N', '4G_coverage'] = 'NO'

coverage.loc[coverage['coverage_type'] == '4', 'coverage_type'] = '4G'
coverage.loc[coverage['coverage_type'] == '3', 'coverage_type'] = '3G'
coverage.loc[coverage['coverage_type'] == '2', 'coverage_type'] = '2G'
coverage.loc[coverage['coverage_type'] == 'N', 'coverage_type'] = 'None'

coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_fc.coverage_type

master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/RW_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
master_w_coverage_connectivity['connectivity'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_availability'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_availability'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_availability'] == 'NO', 'connectivity_type'] = 'None'
master_w_coverage_connectivity.rename(columns={'connectivity_type':'type_connectivity'}, inplace=True)
master_w_coverage_connectivity.to_csv('./data/RW_master_w_coverage_connectivity.csv', encoding='utf-8')

In [36]:
master_w_coverage.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,internet_availability,number_of_students,number_of_teachers,computers,connectivity_type,coverage_availability,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,USB_MODEM,28,1,260.0,3G,YES,4G,YES,NO,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,USB_MODEM,62,3,272.0,3G,YES,4G,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,USB_MODEM,64,4,218.0,3G,YES,4G,YES,NO,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,USB_MODEM,80,2,252.0,3G,YES,4G,YES,YES,NO
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,USB_MODEM,98,3,259.0,3G,YES,4G,YES,YES,YES


In [37]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='None'].head(10)

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,number_of_students,number_of_teachers,computers,type_connectivity,coverage_availability,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
838,cf2e4d25-69c8-4fea-b2a3-016ee28db593,Rwanda,government,G_RWA_00267,441003.0,GS RUTOVU,Secondary,-1.346360,29.823080,886adc3905fffff,...,1009,16,481.0,None,NO,None,NO,NO,NO,NO
2043,b470af04-ba80-4069-a457-bf4562e449d6,Rwanda,government,G_RWA_02063,441004.0,GROUPE SCOLAIRE NYABIZI,Secondary,-1.324830,29.829890,886adc3943fffff,...,244,20,NaN,None,NO,None,NO,NO,NO,NO
2647,6d23b2ce-ebd7-4815-866f-5ee49a2ddf8d,Rwanda,government,G_RWA_02701,440212.0,KILINGA,Primary,-1.362000,29.879056,886adc398dfffff,...,260,20,NaN,None,NO,None,NO,NO,NO,NO
3438,92f51147-d471-4f3f-878c-77221fc71d5f,Rwanda,government,G_RWA_03532,340803.0,GATINDORI,Primary,-1.731806,29.484250,886adeb4e7fffff,...,617,20,NaN,None,NO,None,NO,NO,NO,NO


In [38]:
master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,...,number_of_students,number_of_teachers,computers,type_connectivity,coverage_availability,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
0,cdf4f80c-e3cc-4ebc-8931-8fe76018cc61,Rwanda,government,G_RWA_00002,521312.0,GS MUHORORO,Secondary,-2.159389,29.547972,886ade34e7fffff,...,28,1,260.0,3G,YES,4G,YES,NO,YES,YES
1,a58be9dc-59c4-4cf9-96f5-e5b8fa84ba82,Rwanda,government,G_RWA_00005,331220.0,GS MWENDO,Secondary,-1.730639,30.212722,886ad88707fffff,...,62,3,272.0,3G,YES,4G,YES,YES,YES,YES
2,9fe38ae7-7d51-4646-841b-1b6daa31c987,Rwanda,government,G_RWA_00006,411305.0,GS NYAGISOZI,Secondary,-2.295028,29.645556,886ade2481fffff,...,64,4,218.0,3G,YES,4G,YES,NO,YES,YES
3,8b5c3930-c3ea-44d8-a510-086f8c6889aa,Rwanda,government,G_RWA_00009,510619.0,GS RUGANGO,Secondary,-2.371833,30.599167,886ad821d7fffff,...,80,2,252.0,3G,YES,4G,YES,YES,NO,YES
4,2288686a-e13e-423a-b0bb-b7d08452e9a8,Rwanda,government,G_RWA_00012,130610.0,GS GIHINGA,Secondary,-1.926806,30.591806,886ad8a095fffff,...,98,3,259.0,3G,YES,4G,YES,YES,YES,YES
